In [1]:
import requests
import json
import webbrowser
from datetime import datetime, timedelta
#import pandas

### Step 1: Get the access token

The code checks if the token is still vaid. Otherwise, use the secret_key and secret_id to get a new access key.

In [2]:
#from secrets import gocardless_secret_id, gocardless_secret_key

def get_access_token():
    url = "https://bankaccountdata.gocardless.com/api/v2/token/new/"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    payload = {
        "secret_id": "7bb4840d-9bbc-4f3b-8626-56e05770810f",
        "secret_key": "8468359f502c63c2d92e38e756f41c6c263fb74874a318e20f5ae7e47ade8c20724a7e6345c73cb21d99a1cfb1fb7650e6a0c3dc93df4e09777fcb61f74097f7"
    }
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 200:
        access_response = response.json()
        access_token = access_response.get("access", "")
        access_expiry = access_response.get("access_expires", "")
        if access_token:
            print("Access token obtained successfully")
            return access_token
        else:
            print("Access token not found in the response.")
            return None
    else:
        print(f"Failed to retrieve access token. Status Code: {response.status_code}")
        return None

def has_key_expired(expiry_timestamp):
    current_time = datetime.now()
    expiry_time = datetime.strptime(expiry_timestamp, '%Y-%m-%d %H:%M:%S')
    return current_time >= expiry_time

if __name__ == "__main__":
    access_token = get_access_token()

Access token obtained successfully


### Step 2: Get a list of institution available in the country

In [3]:
    # Step 2: Get a list of institution available in the country
    def fetch_institutions(access_token):
        url = "https://bankaccountdata.gocardless.com/api/v2/institutions/?country=gb"
        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {access_token}"
        }
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            institutions_list = response.json()
            num_institutions = len(institutions_list)
            print(f"Number of institutions obtained: {num_institutions}\n")
            return institutions_list  # Return the entire list of institutions as a list
        else:
            print(f"Failed to retrieve data: {response.status_code}")
            return []

    if __name__ == "__main__":
        institutions = fetch_institutions(access_token)

Number of institutions obtained: 108



### Step 3: Display the list of institutions and prompt user to select one

In [4]:
def select_institution(institutions):
    print("List of Institutions:")
    for index, institution in enumerate(institutions, start=1):
        print(f"{index}. {institution.get('name', '')}, {institution.get('transaction_total_days', '')}")  # Display institution names
    
    # Prompt user to select an institution
    selection = input("Please enter the number corresponding to the institution you wish to select: ")
    
    try:
        index = int(selection) - 1  # Convert user input to index (subtract 1 as index starts from 0)
        if 0 <= index < len(institutions):
            selected_institution = institutions[index]
            print(f"You have selected: {selected_institution.get('name', '')}")
            return selected_institution
        else:
            print("Invalid selection. Please try again.")
            return None
    except ValueError:
        print("Invalid input. Please enter a valid number.")
        return None

if __name__ == "__main__":
    selected_institution = select_institution(institutions)

List of Institutions:
1. ABN AMRO Bank Commercial, 540
2. Airwallex, 730
3. Allied Irish Banks, 730
4. Allied Irish Banks (NI), 730
5. Allied Irish Banks Corporate, 730
6. Alpha FX, 730
7. Amazon card (Newday), 730
8. American Express, 90
9. Aqua card (Newday), 730
10. Arbuthnot Latham, 90
11. Argos card (Newday), 730
12. Bank of Ireland 365 Online, 365
13. Bank of Ireland Business On Line, 365
14. Bank of Scotland Business, 90
15. Bank of Scotland Commercial, 90
16. Bank of Scotland Personal, 90
17. Barclaycard Commercial Payments, 730
18. Barclaycard UK, 730
19. Barclays Business, 730
20. Barclays Corporate, 730
21. Barclays Personal, 730
22. Barclays Wealth, 730
23. BBVA, 730
24. BIP card (Newday), 730
25. Burton card (Newday), 730
26. C. Hoare & Co, 730
27. Capital One, 180
28. Cashplus Bank, 540
29. Cater Allen, 90
30. Caxton, 730
31. Chase Bank, 395
32. Chelsea Building Society, 730
33. Coutts&Co, 730
34. Cumberland Building Society, 730
35. Cynergy Bank, 730
36. Danske Bank Busi

### Step 4: Create end-user agreement
This step is optional

In [5]:
def create_end_user_agreement(access_token, selected_institution):
    url = "https://bankaccountdata.gocardless.com/api/v2/agreements/enduser/"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    payload = {
        "institution_id": selected_institution.get("id", ""),
        "max_historical_days": selected_institution.get('transaction_total_days', ''),
        "access_valid_for_days": "90",
        "access_scope": ["balances", "details", "transactions"]
    }
    
    response = requests.post(url, headers=headers, json=payload)
    
    if response.status_code == 201:
        print("Successfully created end-user agreement.")
        user_agreement = response.json()
        agreement_id = user_agreement.get("id", "")
        access_valid_for_days = user_agreement.get("access_valid_for_days", "")
        print(f"The agreement ID is: {agreement_id} and it is valid for {access_valid_for_days} days")
        return user_agreement  # Return the JSON response
    else:
        print(f"Failed to create end-user agreement. Status Code: {response.status_code}")
        return None

if __name__ == "__main__":
    user_agreement = create_end_user_agreement(access_token, selected_institution)

Successfully created end-user agreement.
The agreement ID is: 6be2b97d-e88d-46d5-8272-d7ff287f890b and it is valid for 90 days


### Step 5: Create a requisition, which is a collection of inputs for creating links and retrieving accounts

In [6]:
sandbox_id = "SANDBOXFINANCE_SFIN0000"

def create_requisition(access_token, selected_institution, user_agreement):
    url = "https://bankaccountdata.gocardless.com/api/v2/requisitions/"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with the actual access token
    }
    
    payload = {
        "redirect": "https://sparkcharitable.framer.website/",
        "institution_id": selected_institution.get("id", ""),
        # not needed, will need to be unique string 
        #"reference": "124181",
        "agreement": user_agreement.get("id", ""),
        "user_language": "EN"
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 201:
        print("Successfully created requisition.")
        requisition_data = response.json()
        redirect = requisition_data.get("link", "")
        webbrowser.open(redirect)
        print("Requisition Data:", requisition_data)
        return requisition_data
    else:
        print(f"Failed to create requisition. Status Code: {response.status_code}")
        return None

if __name__ == "__main__":
    requisition_data = create_requisition(access_token, selected_institution, user_agreement)
    requisition_id = requisition_data.get("id", "")

Successfully created requisition.
Requisition Data: {'id': 'a7f5d4b1-ef16-406f-8115-f70d9067ace2', 'created': '2023-12-26T19:43:03.055829Z', 'redirect': 'https://sparkcharitable.framer.website/', 'status': 'CR', 'institution_id': 'MONZO_MONZGB2L', 'agreement': '6be2b97d-e88d-46d5-8272-d7ff287f890b', 'reference': 'a7f5d4b1-ef16-406f-8115-f70d9067ace2', 'accounts': [], 'user_language': 'EN', 'link': 'https://bankaccountdata.gocardless.com/psd2/start/a7f5d4b1-ef16-406f-8115-f70d9067ace2/MONZO_MONZGB2L', 'ssn': None, 'account_selection': False, 'redirect_immediate': False}


### Step 6: The user's bank accounts is listed once the requisition is completed

In [7]:
def get_requisition_details(access_token, requisition_id):
    url = f"https://bankaccountdata.gocardless.com/api/v2/requisitions/{requisition_id}/"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}" 
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        print("Successfully retrieved account data")
        requisition_data = response.json()
        return requisition_data
    
    else:
        print(f"Failed to retrieve requisition details. Status Code: {response.status_code}")
        return None
    
if __name__ == "__main__":
    requisition_data = get_requisition_details(access_token, requisition_id)
    account_list = requisition_data.get("accounts", "")
    print (account_list)

Successfully retrieved account data
[]


### Step 8: Get all account details

In [8]:
def fetch_account_details(access_token, account_list):
    base_url = "https://bankaccountdata.gocardless.com/api/v2/accounts/{}/details/"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"  
    }
    
    # Dictionary to store all account details
    accounts = {}
    accounts['accounts'] = []

    # Loop through each account ID to fetch account details
    for account_id in account_list:
        url = base_url.format(account_id)
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            print(f"Successfully fetched details for account: {account_id}")
            account_details = response.json()
            account_info = {}
            account_info = account_details.get("account", {})
            account_info['id'] = account_id
            accounts['accounts'].append(account_info)
        
        else:
            print(f"Failed to fetch details for account: {account_id}. Status Code: {response.status_code}")
    
    
    formatted_accounts = json.dumps(accounts, indent=4)
    
    print("Account Details:")
    print(formatted_accounts)
    return accounts

if __name__ == "__main__":
    accounts = fetch_account_details(access_token, account_list)

Account Details:
{
    "accounts": []
}


### Step 9: Prompt user to select a bank account 

In [9]:
def select_account(accounts):
    
    # Step 2: Display account details with numbers for user selection
    for index, (account_details) in enumerate(accounts["accounts"], start=1):
        owner_name = account_details['ownerName']
        bban = account_details['bban']
        currency = account_details['currency']
        print(f"{index}. Owner: {owner_name}, {currency}, BBAN: {bban}")
    
    # Step 3: Prompt user for input
    selection = input("Please enter the number corresponding to the account you wish to select: ")
    
    try:
        index = int(selection) - 1  # Convert user input to index (subtract 1 as index starts from 0)
        
        # Step 4: Validate user input and return selected account details
        if 0 <= index < len(accounts):
            selected_account_details = accounts["accounts"][index]
            selected_account_id = selected_account_details["id"]  # Get the resourceId from the selected account
            print(f"You have selected Account ID: {selected_account_id}")
            return selected_account_details
        else:
            print("Invalid selection. Please try again.")
            return None
    except ValueError:
        print("Invalid input. Please enter a valid number.")
        return None

if __name__ == "__main__":
    selected_account = select_account(accounts)
    account_id = selected_account["id"]

### Step 10: Access account transactions

In [ ]:
def fetch_account_transactions(access_token, account_id):
    # Step 2: Define API Endpoint and Headers
    url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{account_id}/transactions/"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    # Step 3: Make the GET Request
    response = requests.get(url, headers=headers)
    
    # Step 4: Process the API Response
    if response.status_code == 200:
        print("Successfully fetched account transactions.")
        transaction_data = response.json()
        
        # Convert the transactions data to a formatted JSON string
        formatted_json = json.dumps(transaction_data, indent=4)
        
        # Display the transactions data in JSON format
        print(formatted_json)
        
        return transaction_data
    else:
        print(f"Failed to fetch account transactions. Status Code: {response.status_code}")
        return None

if __name__ == "__main__":
    transaction_data = fetch_account_transactions(access_token, account_id)


Successfully fetched account transactions.
{
    "transactions": {
        "booked": [
            {
                "transactionId": "tx_0000Ad1b3sn97ypqXx0XUf",
                "bookingDate": "2023-12-21",
                "bookingDateTime": "2023-12-21T05:44:43.102Z",
                "transactionAmount": {
                    "amount": "12.0000",
                    "currency": "GBP"
                },
                "debtorName": "Companies House",
                "remittanceInformationUnstructured": "Companies House",
                "proprietaryBankTransactionCode": "mastercard",
                "merchantCategoryCode": "9399",
                "internalTransactionId": "e2e820c41750a26058d271ec57df6a2f"
            },
            {
                "transactionId": "tx_0000AcyaoBvdctPfKxMmX4",
                "bookingDate": "2023-12-19",
                "bookingDateTime": "2023-12-19T18:57:44.918Z",
                "transactionAmount": {
                    "amount": "-12.0000",
   

### Step 11: Load the list of charity

In [ ]:
source_file = r"C:\Users\abell\Spark\backend\api\publicextract.charity.json"

with open(source_file, 'r', encoding='utf-8-sig') as file:
    list_of_charities = json.load(file)
    charity_list = [{"charity_name": charity["charity_name"], "registered_charity_number": charity["registered_charity_number"], "linked_charity_number": charity["linked_charity_number"]} for charity in list_of_charities]
    print(charity_list[:10])

[{'charity_name': 'MISSION ROOM AND TRUST', 'registered_charity_number': 200027, 'linked_charity_number': 1}, {'charity_name': 'HITCHAM FREE CHURCH', 'registered_charity_number': 200027, 'linked_charity_number': 2}, {'charity_name': 'TOWN LANDS CHARITY FOR THE POOR', 'registered_charity_number': 200028, 'linked_charity_number': 1}, {'charity_name': 'TOWN LANDS CHARITY FOR THE CHURCH', 'registered_charity_number': 200028, 'linked_charity_number': 2}, {'charity_name': 'CLOPHILL RELIEF IN NEED CHARITY', 'registered_charity_number': 200034, 'linked_charity_number': 1}, {'charity_name': 'MARL ALLOTMENT', 'registered_charity_number': 200034, 'linked_charity_number': 5}, {'charity_name': "GEORGE DUNCUMB'S CHARITY FOR THE PARISH CLERK", 'registered_charity_number': 200037, 'linked_charity_number': 1}, {'charity_name': "HARRIET GROTE'S ECCLESIASTICAL CHARITY", 'registered_charity_number': 200037, 'linked_charity_number': 2}, {'charity_name': 'CHARLES HAMMOND (OTHERWISE PALMER)', 'registered_cha

### Step 12: Match the transactions with the charities

In [ ]:
def search_by_charity_name(transaction_data, charity_list):
    eligible_transactions = []
    eligible_charity = []
    transaction_types = transaction_data["transactions"].keys()

    for transaction_type in transaction_types:
        transactions = transaction_data["transactions"][transaction_type]
        for transaction in transactions:
            if "remittanceInformationUnstructured" in transaction:
                transaction_description = transaction["remittanceInformationUnstructured"].lower()
                for charity in charity_list:
                    if charity['charity_name'].lower() in transaction_description:
                        eligible_transactions.append(transaction)
                        eligible_charity.append(charity['charity_name'])
    
    
    print (eligible_charity)
    return eligible_transactions, eligible_charity

if __name__ == "__main__":
    eligible_transactions = search_by_charity_name(transaction_data, charity_list)
    print (eligible_transactions)

['US', 'US', 'US', 'IA', 'BARB', 'IA', 'US', 'LIVE', 'HENRY', 'US', 'TEY', 'US', 'US', 'US', 'US', 'US', 'US', 'TEY', 'US', 'CLIMB', 'IA', 'PORT', 'US', 'REPUBLIC', 'CLIMB', 'IA', 'US', 'IA', 'IA', 'IA', 'TEY', 'CLIMB', 'US', 'US', 'IA', 'CAPITAL', 'US', 'BLEND']
([{'transactionId': 'tx_0000Ad1b3sn97ypqXx0XUf', 'bookingDate': '2023-12-21', 'bookingDateTime': '2023-12-21T05:44:43.102Z', 'transactionAmount': {'amount': '12.0000', 'currency': 'GBP'}, 'debtorName': 'Companies House', 'remittanceInformationUnstructured': 'Companies House', 'proprietaryBankTransactionCode': 'mastercard', 'merchantCategoryCode': '9399', 'internalTransactionId': 'e2e820c41750a26058d271ec57df6a2f'}, {'transactionId': 'tx_0000AcyaoBvdctPfKxMmX4', 'bookingDate': '2023-12-19', 'bookingDateTime': '2023-12-19T18:57:44.918Z', 'transactionAmount': {'amount': '-12.0000', 'currency': 'GBP'}, 'creditorName': 'Companies House', 'remittanceInformationUnstructured': 'Companies House', 'proprietaryBankTransactionCode': 'mast